# Tracing Without LangChain

LangSmith lets you instrument **any LLM application,** no LangChain required. This can help you debug, evaluate, and monitor your app without having to learn any particular framework's unique semantics. LangSmith provides two primary ways to do this in python (there currently is just the `RunTree` for js):

1. Using the `@traceable` decorator
2. Using a `RunTree`

In the following walkthrough, you will create an example app using the traceable decorator and explore some configurations you can do for logging auxiliary information. Then you will rewrite the app using the RunTree for more control.

The chat app used in this walkthrough generates an argument about a topic. It is composed of three separate components, which will all show up in the traces:

1. An argument generation function
2. Critique function
3. Refine function

Each of these "chain" functions will call an openai `llm` function, and the whole series of calls will itself be organized beneath a parent `argument_chain` function.

Put together, the program will generate a trace like the following:

![RunTree 1](img/snapshot_1.png)

Now that we know what we're building, let's get started!

## Prerequisites

Let's first install some required packages. We'll need both the langsmith SDK and the OpenAI SDK.

In [1]:
%pip install -U langsmith > /dev/null
%pip install -U openai > /dev/null

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Next, configure the API Key in the environment to make sure traces are logged to your account.

In [2]:
# %env LANGCHAIN_API_KEY=<your-api-key>
# %env LANGCHAIN_PROJECT=tracing-cookbook-tutorial

## Using the decorator

Next, define your chat application. Use the `@traceable` decorator to automatically instrument your
function calls.

In [3]:
from datetime import datetime
from typing import List, Optional, Tuple

import openai
from langsmith.run_helpers import traceable


@traceable(run_type="llm")
def call_openai(data: List[dict], model: str = "gpt-3.5-turbo", temperature: float = 0.0):
    return openai.ChatCompletion.create(
        model=model,
        messages=data,
        temperature=temperature,
    )


@traceable(run_type="chain")
def argument_generator(query: str, additional_description: str = "") -> str:
    return call_openai(
        [
            {"role": "system", "content": f"You are a debater making an argument on a topic."
             f"{additional_description}"
             f" The current time is {datetime.now()}"},
            {"role": "user", "content": f"The discussion topic is {query}"}
        ]
    ).choices[0].message.content


@traceable(run_type="chain")
def critic(argument: str) -> str:
    return call_openai(
        [ 
            {"role": "system", "content": f"You are a critic."
           "\nWhat unresolved questions or criticism do you have after reading the following argument?"
            "Provide a concise summary of your feedback."},
            {"role": "system", "content": argument}
            
        ]
    ).choices[0].message.content


@traceable(run_type="chain")
def refiner(query: str, additional_description: str, current_arg: str, criticism: str) -> str:
    return call_openai(
        [
            {"role": "system", "content": f"You are a debater making an argument on a topic."
             f"{additional_description}"
             f" The current time is {datetime.now()}"},
            {"role": "user", "content": f"The discussion topic is {query}"},
            {"role": "assistant", "content": current_arg},
            {"role": "user", "content": criticism},
            {"role": "system", "content": "Please generate a new argument that incorporates the feedback from the user."}
        ]
    ).choices[0].message.content
    

@traceable(run_type="chain")      
def argument_chain(query: str, additional_description: str = "") -> str:
    argument = argument_generator(query, additional_description)
    criticism = critic(argument)
    return refiner(query, additional_description, argument, criticism)

Now call the chain. If you set up your API key correctly at the start of this notebook, all the results should be traced to  [LangSmith](https://www.smith.langchain.com).

In [4]:
result = argument_chain(
    "Whether sunshine is good for you.", 
    additional_description="Provide a concise, few sentence argument on why sunshine is good for you.",
)
print(result)

While it is true that excessive sun exposure can lead to sunburn, skin damage, and an increased risk of skin cancer, it is important to acknowledge that moderate and responsible exposure to sunshine can have significant health benefits. Sunlight is a natural source of vitamin D, which plays a crucial role in bone health, immune function, and mental well-being. While dietary supplements can provide vitamin D, they may not be as effective as sunlight in raising vitamin D levels. By practicing sun safety measures, such as wearing sunscreen and seeking shade during peak hours, individuals can enjoy the benefits of sunshine while minimizing the potential risks.


## Working with runs

The above is all you need to save your app traces to LangSmith! The decorator handles the call relationships for you in a background thread to avoid interfering with your program execution.

Beyond tracing, you may want to use LangSmith for other things like monitoring user feedback. You can view the run information by looking at the `RunTree` created for that event. To do so, change any wrapped function's signature to accept a `run_tree` argument. Then the `@traceable` decorator will inject the current run tree object into the wrapped function. This can be useful if you want to:
- Add user feedback to the run
- Inspect or save the run ID or its parent
- Manually log child runs or their information to another destination
- Explicitly pass the run tree to other functions that may be called within thread or process pools (or on separate machines) to maintain trace cohesion

Below, our `argument_chain2` function is identical to the previous one but views the injected run_tree so we can fetch the latest run.

In [5]:
from langsmith import RunTree
    
run_ids = []
    
@traceable(run_type="chain")      
def argument_chain2(query: str, *, additional_description: str = "", run_tree: RunTree) -> str:
    argument = argument_generator(query, additional_description)
    run_ids.append(run_tree.id)
    criticism = critic(argument)
    # If you want to manually provide the run tree to a 'traceable' function,
    # You can pass it via the `langsmith_extra` arguments. This is optional!
    return refiner(query, additional_description, argument, criticism, langsmith_extra={"run_tree": run_tree})       

In [6]:
result = argument_chain2(
    "Whether sunshine is good for you.", 
    additional_description="Provide a concise, few sentence argument on why sunshine is good for you.",
)

View the run ID that you've collected and then create a public link to share. You could also add the ID to other logs or instrumentation in your app.

In [7]:
from langsmith import Client

client = Client()

In [9]:
client.create_feedback(
    run_ids[-1],
    "user_feedback",
    score=0.5,
    correction={"generation": "Sunshine is nice. Full stop."},
)

Feedback(id=UUID('0979fb2b-aed7-45d8-b59a-7e64fc3f6b81'), created_at=datetime.datetime(2023, 8, 8, 22, 28, 12, 898173), modified_at=datetime.datetime(2023, 8, 8, 22, 28, 12, 898177), run_id=UUID('30fc332a-2810-48aa-87d3-b3a2e22e278c'), key='user_feedback', score=0.5, value=None, comment=None, correction={'generation': 'Sunshine is nice. Full stop.'}, feedback_source=FeedbackSourceBase(type='api', metadata=None))

In [10]:
shared_link = client.share_run(run_ids[-1])
shared_link

'https://dev.smith.langchain.com/public/3138fe8d-e131-4359-bc0a-c0bef67d92f9/r'

![RunTree 2](./img/snapshot_2.png)

## Configuring Traces

One way to make your application traces more useful or actionable is to tag or add metadata to the logs. That way you can do things like track the version of your code or deployment environment in a single project.

The traceable decorator can be configured to add additional information such as:
- string tags
- arbitrary key-value metadata
- custom trace names
- manually-specified run ID

Below is an example. 

In [11]:
# You can add tags and metadata (or even the project name) directly in the decorator
@traceable(run_type="chain", name="My Argument Chain", tags=["tutorial"], metadata={"githash": "e38f04c83"})      
def argument_chain_3(query: str, run_tree: RunTree, additional_description: str = "") -> str:
    argument = argument_generator(query, additional_description)
    run_ids.append(run_tree.id)
    criticism = critic(argument)
    # Passing the run tree via `langsmith_extra` is optional. It is automatically tracked in the background.
    return refiner(query, additional_description, argument, criticism, langsmith_extra={"run_tree": run_tree})       

In [12]:
from uuid import uuid4

requested_uuid = uuid4()

result = argument_chain_3(
    "Whether sunshine is good for you.", 
    additional_description="Provide a concise, few sentence argument on why sunshine is good for you.",
    # You can also add tags, metadata, or the run ID directly via arguments to the langsmith_extra argument
    # at all-time.
    langsmith_extra={
        "tags": ["another-tag"],
        "metadata": {"another-key": 1},
        "run_id": requested_uuid,
    }
)

In [14]:
# We can log feedback for the run directly since we've controlled the ID it's assuming
client.create_feedback(
    run_ids[-1],
    "user_feedback",
    score=1,
    source_info={"origin": "example notebook"}

)

Feedback(id=UUID('b99ccbb0-d33e-4106-aed8-34b50929d544'), created_at=datetime.datetime(2023, 8, 8, 22, 29, 51, 484809), modified_at=datetime.datetime(2023, 8, 8, 22, 29, 51, 484813), run_id=UUID('8f7a79b7-8c36-4b8f-911e-716e63913e0f'), key='user_feedback', score=1.0, value=None, comment=None, correction=None, feedback_source=FeedbackSourceBase(type='api', metadata={'origin': 'example notebook'}))

In [15]:
shared_link = client.share_run(requested_uuid)
shared_link

'https://dev.smith.langchain.com/public/e06d142a-fd34-4479-9a06-4cd38200100d/r'

You can see the tags are saved to the run linked above.

[![Tagged Run Tree](./img/snapshot_3.png)](https://dev.smith.langchain.com/public/502a9bc4-44b1-4e86-bb19-3d6b9766f248/r)

Clicking in to the 'Metadata' tab, you can see the metadata has been stored for the trace.

[![Run Tree Metadata](./img/snapshot_3_metadata.png)](https://dev.smith.langchain.com/public/502a9bc4-44b1-4e86-bb19-3d6b9766f248/r?tab=2)

Once you've stored these tagged runs, you can filter and search right in the web app by clicking on the suggested filters or by writing out the query in the search bar:


![Filtering Tags](./img/snapshot_3_tag_filter.png)

## Manual tracing using the RunTree

The `traceable` decorator is a lightweight and extremely easy way to instrument your app and stream traces to LangSmith. It is less flexible if you want to control things like the schema for how the logs are saved or how errors are displayed. Below, we will rewrite the chat application in a way that explicitly uses the run tree to save traces.

In [16]:
from langsmith import RunTree

def call_openai(data: List[dict], run_tree: RunTree,  model: str = "gpt-3.5-turbo", temperature: float = 0.0):
    child = run_tree.create_child(
        name="ChatOpenAI",
        inputs={"data": data, "model": model, "temperature": temperature},
        run_type="llm",
    )
    child.post()
    try:
        result = openai.ChatCompletion.create(
            model=model,
            messages=data,
            temperature=temperature,
        )
        child.end(outputs={"response": result})
    except Exception as e:
        child.end(error=str(e))
        raise
    finally:
        child.patch()
    return result


def argument_generator(query: str, run_tree: RunTree, additional_description: str = "") -> str:
    child = run_tree.create_child(name="ArgumentGenerator", inputs={"query": query, additional_description: additional_description}, run_type="chain")
    child.post()
    try:
        result = call_openai(
            [
                {"role": "system", "content": f"You are a debater making an argument on a topic."
                    f"{additional_description}"
                    f" The current time is {datetime.now()}"},
                {"role": "user", "content": f"The discussion topic is {query}"}
            ], run_tree=child
        ).choices[0].message.content
        child.end(outputs={"argument": result})
    except Exception as e:
        child.end(error=str(e))
        raise
    finally:
        child.patch()
    return result


def critic(argument: str, run_tree: RunTree) -> str:
    child = run_tree.create_child(name="Critic", run_type="chain", inputs={"argument": argument})
    child.post()
    try:
        result = call_openai(
            [
                {"role": "system", "content": f"You are a critic."
                    "\nWhat unresolved questions or criticism do you have after reading the following argument?"
                    "Provide a concise summary of your feedback."},
                {"role": "system", "content": argument}
            ], run_tree=child
        ).choices[0].message.content
        child.end(outputs={"criticism": result})
    except Exception as e:
        child.end(error=str(e))
        raise
    finally:
        child.patch()
    return result


def refiner(query: str, additional_description: str, current_arg: str, criticism: str, run_tree: RunTree) -> str:
    child = run_tree.create_child(name="Refiner", run_type="chain", inputs={"query": query, "additional_description": additional_description})
    child.post()
    try:
        result = call_openai(
            [
                {"role": "system", "content": f"You are a debater making an argument on a topic."
                    f"{additional_description}"
                    f" The current time is {datetime.now()}"},
                {"role": "user", "content": f"The discussion topic is {query}"},
                {"role": "assistant", "content": current_arg},
                {"role": "user", "content": criticism},
                {"role": "system", "content": "Please generate a new argument that incorporates the feedback from the user."}
            ], run_tree=child
        ).choices[0].message.content
        child.end(outputs={"refined_argument": result})
    except Exception as e:
        child.end(error=str(e))
        raise
    finally:
        child.patch()
    return result
    

run_tree_run_ids = []
def argument_chain4(query: str, additional_description: str = "", run_tree: RunTree = None) -> str:
    inputs = {
        "query": query,
        "additional_description": additional_description
    }
    if run_tree is None:
        argument_run_tree = RunTree(name="ArgumentChain", run_type="chain", inputs=inputs)
    else:
        argument_run_tree = run_tree.create_child(name="ArgumentChain", run_type="chain", inputs=inputs)
    argument_run_tree.post()
    run_tree_run_ids.append(argument_run_tree.id)
    try:
        argument = argument_generator(query, additional_description=additional_description, run_tree=argument_run_tree)
        criticism = critic(argument, run_tree=argument_run_tree)
        result = refiner(query, additional_description, argument, criticism, run_tree=argument_run_tree)
        argument_run_tree.end(outputs={"final_argument": result})
    except Exception as e:
        argument_run_tree.end(error=str(e))
        raise
    finally:
        argument_run_tree.patch()
    return result

In [17]:
result = argument_chain4(
    "Whether sunshine is good for you.", 
    additional_description="Provide a concise, few sentence argument on why sunshine is good for you.",
)

In [18]:
latest_id = run_tree_run_ids[-1]
shared_link = client.share_run(latest_id)
shared_link

'https://dev.smith.langchain.com/public/091ebe86-f137-4079-ae8f-6db01e367801/r'

![Manual Run Tree](./img/snapshot_4.png)